In [ ]:
import numpy as np
import pandas as pd


# Recreate the modified file (since it doesn't exist in current session)
data_path = "Data/Data.csv"
salaries_path = "Data/Salaries.csv"

data_df = pd.read_csv(data_path)
salaries_df = pd.read_csv(salaries_path)

# Clean Salaries.csv
salaries_df = salaries_df.drop(columns=['Id', 'EmployeeName', 'Notes', 'Agency', 'Status', 'Year'])
salaries_df['BasePay'] = pd.to_numeric(salaries_df['BasePay'], errors='coerce')
salaries_df['OvertimePay'] = pd.to_numeric(salaries_df['OvertimePay'], errors='coerce')
salaries_df['OtherPay'] = pd.to_numeric(salaries_df['OtherPay'], errors='coerce')
salaries_df['Benefits'] = pd.to_numeric(salaries_df['Benefits'], errors='coerce')
salaries_df = salaries_df[salaries_df['TotalPayBenefits'] > 0]

# Clean Data.csv
data_df = data_df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'EstimatedSalary'])

# Assign occupations randomly
np.random.seed(42)
data_df['Occupation'] = np.random.choice(salaries_df['JobTitle'].unique(), size=len(data_df))

# Merge mean salary
job_salaries = salaries_df.groupby('JobTitle')['TotalPayBenefits'].mean().reset_index()
job_salaries.rename(columns={'JobTitle': 'Occupation', 'TotalPayBenefits': 'MeanSalary'}, inplace=True)
data_df = data_df.merge(job_salaries, on='Occupation', how='left')

# Feature engineering
data_df['Age_Tenure_Score'] = (data_df['Age'] * data_df['Tenure'])
data_df['Age_Tenure_Score'] = (data_df['Age_Tenure_Score'] - data_df['Age_Tenure_Score'].min()) / (data_df['Age_Tenure_Score'].max() - data_df['Age_Tenure_Score'].min())
data_df['Credit_Score_Norm'] = (data_df['CreditScore'] - data_df['CreditScore'].min()) / (data_df['CreditScore'].max() - data_df['CreditScore'].min())
data_df['Balance_Norm'] = (data_df['Balance'] - data_df['Balance'].min()) / (data_df['Balance'].max() - data_df['Balance'].min())

# Realistic salary calculation
data_df['RealisticSalary'] = (data_df['Age_Tenure_Score'] * 0.4 +
                              data_df['Credit_Score_Norm'] * 0.3 +
                              data_df['Balance_Norm'] * 0.3) * 200000 + 50000

# Drop helper columns but keep Occupation
data_df = data_df.drop(columns=['Age_Tenure_Score', 'Credit_Score_Norm', 'Balance_Norm', 'MeanSalary'])

# Save to CSV
output_path = "Data/Modified_Data.csv"
data_df.to_csv(output_path, index=False)

C:\Users\DELL\AppData\Local\Temp\ipykernel_4952\2035950234.py:9: DtypeWarning: Columns (3,4,5,6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  salaries_df = pd.read_csv(salaries_path)
